<a href="https://colab.research.google.com/github/Maxxx-VS/Test_task/blob/master/Sushi_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тестовое задание

Постановка задачи:
В оффлайн-продажах важно оценивать предполагаемый поток клиентов и объем проданной продукции. В рамках данного тестового задания необходимо выполнить следующие шаги:
1.	На основании предоставленных исходных данных выберите и сформируйте список уникальных номенклатурных позиций, которые присутствуют в продажах.
2.	Подготовьте «силуэт продаж» – выявите и опишите характерный профиль продаж по каждой номенклатурной позиции, учитывая сезонность по дням недели и месяцам. Силуэт должен отражать типичные изменения объема продаж во времени.
3.	Используя сформированный силуэт продаж, подготовьте предварительный прогноз объема продаж на завтрашний день для каждой номенклатурной позиции (завтрашним считается день, следующий за днем сдачи тестового задания).
4.	С помощью инструментов машинного обучения проведите пост-обработку и корректировку предварительного прогноза, полученного по силуэту продаж, до более точного значения.
Предоставьте итоговый результат в виде таблицы с перечнем уникальных номенклатурных позиций и соответствующим прогнозом продаж. Дополнительно кратко опишите используемый подход и методику расчета и корректировки прогноза.
Из 3 представленных на выбор файлов (данные за прошлый год по трем разным торговым точкам), выбрать нужно только один (в решении можно использовать все три).

Описание переменных:
CloseTime – время чека
DishAmountInt – количество позиций (целевая переменная)
DishDiscountSumInt – сумма в рублях
DishName – название блюда
OpenDate.Typed– дата заказа
RestorauntGroup– название торговой точки

Рекомендации:
Цель данного тестового посмотреть на ход решения и рассуждений.
Укажите, какие переменные используются для построения силуэта продаж и модели, почему именно они, какие подходы используются в решении (приветствуется добавление собственных признаков). Никаких ограничений в плане выбора технологий прогнозирования нет (кроме, конечно, здравого смысла). Как итог нужно презентовать код решения с пояснениями (возможно, ноутбук в google colab или jupyter) и ответить на возникающие вопросы. Готовить слайды рр на данном этане не нужно.


In [3]:
# ЗАГРУЗКА ДАННЫХ

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Sushi_test_task/id_точки 16.xlsx'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# СОЗДАЁМ ДАТАФРЕЙМ

import pandas as pd
df = pd.read_excel(path)

In [20]:
# ПРОСМОТР РАЗМЕРНОСТИ ДАТАФРЕЙМА

df.shape

(326619, 6)

In [21]:
# ВЫВЕДЕМ НАИМЕНВАНИЕ ВСЕХ СТОЛБЦОВ

df.columns

Index(['CloseTime', 'DishAmountInt', 'DishDiscountSumInt', 'DishName',
       'RestorauntGroup', 'OpenDate.Typed'],
      dtype='object')

In [22]:
# СТРУКТУРА ДАТАФРЕЙМА И ПРОВЕРКА ПРОПУСКОВ

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326619 entries, 0 to 326618
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CloseTime           326619 non-null  datetime64[ns]
 1   DishAmountInt       326619 non-null  int64         
 2   DishDiscountSumInt  326619 non-null  int64         
 3   DishName            326619 non-null  object        
 4   RestorauntGroup     326619 non-null  object        
 5   OpenDate.Typed      326619 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 15.0+ MB


In [23]:
# ПРОСМОТР ДАННЫХ

df.head(3)

,CloseTime,DishAmountInt,DishDiscountSumInt,DishName,RestorauntGroup,OpenDate.Typed
0,2025-02-08 17:16:49,6,228,Лава МСК рол,"Омск, АТ-Маркет, Дианова, 14",2025-02-08
1,2025-02-08 10:17:08,1,5,ПАКЕТ МАЙКА ОДНОР,"Омск, АТ-Маркет, Дианова, 14",2025-02-08
2,2025-02-08 10:17:08,1,25,Контейнер большой одн,"Омск, АТ-Маркет, Дианова, 14",2025-02-08


In [24]:
# ХАРАКТЕРИСТИКИ ДАТАФРЕЙМА

df.describe()

,CloseTime,DishAmountInt,DishDiscountSumInt,OpenDate.Typed
count,326619,326619.000000,326619.000000,326619
mean,2024-08-14 22:06:43.376683776,2.549610,160.189435,2024-08-14 05:28:06.996776448
min,2024-02-01 10:03:47,0.000000,0.000000,2024-02-01 00:00:00
25%,2024-05-10 17:31:41,1.000000,13.000000,2024-05-10 00:00:00
50%,2024-08-16 15:27:35,1.000000,82.000000,2024-08-16 00:00:00
75%,2024-11-15 16:27:12,3.000000,190.000000,2024-11-15 00:00:00
max,2025-02-28 21:17:26,60.000000,6500.000000,2025-02-28 00:00:00
std,NaN,2.699514,240.291049,NaN


In [58]:
# ПЕРЕИМЕНУЕМ СТОЛБЕЦ ДЛЯ ПРАВИЛЬНОЙ АДРЕСАЦИИ К ЕГО НАЗВАНИЮ

df = df.rename(columns={'OpenDate.Typed': 'OpenDate_Typed'})

In [59]:
# УЧИТЫВАЯ, ЧТО ПРЕДОСТАВЛЕННЫЕ ДАННЫЕ ОТНОСЯТСЯ К ОДНОЙ ТОРГОВОЙ ТОЧКЕ - УДАЛЯЕМ ПРИЗНАК "RestorauntGroup"

df = df.drop("RestorauntGroup", axis=1)

KeyError: "['RestorauntGroup'] not found in axis"

In [60]:
# ПРОВЕРКА НЕСОВПАДЕНИЯ ТИПА ДАННЫХ В СТОЛБЦЕ

def check_dtype(column):
    first_type = type(column.iloc[0])
    return all(isinstance(x, first_type) for x in column)

column_CloseTime = check_dtype(df['CloseTime'])
column_DishAmountInt = check_dtype(df['DishAmountInt'])
column_DishDiscountSumInt = check_dtype(df['DishDiscountSumInt'])
column_OpenDate_Typed = check_dtype(df['OpenDate_Typed'])

print(f'Столбец {"CloseTime"} содержит одинаковые типы данных {column_CloseTime}')



Столбец CloseTime содержит одинаковые типы данных True


In [50]:


reference = {
    'CloseTime': type_1,
    'DishAmountInt': type_2,
    'DishDiscountSumInt': type_3,
    'OpenDate.Typed': type_4,
}

def check_dtype():
    for col, ref in reference.items():
        if not isinstance(df[col], ref):
            print(f'Столбец {col} имеет более одного типа данных: {df[col].dtype}')
        else:
            print(f'Столбец {col} содержит одинаковый тип данных')

check_dtype()

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [ ]:
# УДАЛЯЕМ СТРОКИ С РАЗНЫМИ ТИПАМИ ДАННЫХ В СТОЛБЦЕ

def del_dtype():
    for col in df.columns:
        if df[col].dtype != reference:


In [30]:
# ПОВТОРНО ПРОВЕРЯЕМ НЕСОВПАДЕНИЯ ТИПА ДАННЫХ В СТОЛБЦЕ

check_dtype()

Столбец CloseTime содержит одинаковый тип данных
Столбец DishAmountInt имеет более одного типа данных: int64
Столбец DishDiscountSumInt имеет более одного типа данных: int64
Столбец DishName имеет более одного типа данных: object
Столбец RestorauntGroup имеет более одного типа данных: object
Столбец OpenDate.Typed содержит одинаковый тип данных
